In [ ]:
import polars as pl
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures,
)
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError as RMSE, R2Score

In [ ]:
def get_data(poly_features: int = 1):
    # Get Data
    data = pl.read_parquet("data.parquet")
    data = data.drop(["Step", "Light_ID", "Lane", "Intersection_u", "Sim_ID"])
    data = data.with_columns(pl.col("Is_Entrypoint").cast(pl.Int8))
    print(f"Data: {data.shape}")
    print(f"{data.collect_schema()}")

    # Split Data
    X = data.drop("Num_Cars").to_numpy()
    y = data.select(pl.col("Num_Cars")).to_numpy()
    y = y.ravel()
    print("")
    print(f"X: {X.shape}")
    print(f"y: {y.shape}")

    # Scale
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Polynomial Features
    if poly_features > 1:
        poly = PolynomialFeatures(degree=poly_features)
        X = poly.fit_transform(X)

    # Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.6, test_size=0.4, random_state=42
    )
    print("")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"y_test {y_test.shape}")

    # Train Test Validation Split
    X_test, X_val, y_test, y_val = train_test_split(
        X_test, y_test, train_size=0.5, test_size=0.5, random_state=42
    )
    print("")
    print(f"X_test: {X_test.shape}")
    print(f"X_val: {X_val.shape}")
    print(f"y_test: {y_test.shape}")
    print(f"y_val: {y_val.shape}")

    return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
X_train, X_test, X_val, y_train, y_test, y_val = get_data()

In [ ]:
model = Sequential(
    [
        Dense(128, activation="relu"),
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),
        Dense(1),
    ]
)
model.build(input_shape=(None, X_train.shape[1]))

model.compile(
    optimizer=Adam(learning_rate=0.001), loss="mse", metrics=[RMSE(), R2Score()]
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    verbose=1,
)

loss, rmse, r2_score = model.evaluate(X_test, y_test, verbose=1)

print("")
print(f"Loss: {loss}")
print(f"RMSE: {rmse}")
print(f"R2Score: {r2_score}")